In [1]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re
from sklearn import preprocessing

In [7]:
full_df = pd.read_csv(r"shuffled-noIPV6.csv")
# df = pd.read_csv("truncated_training.csv")

print(f"[*] Shape of dataset: {full_df.shape}")
from sklearn.utils import shuffle
full_df.drop(full_df.columns[0], axis=1, inplace=True)
full_df.drop(full_df.columns[13], axis=1, inplace=True) #drop category column
full_df = shuffle(full_df)

print(full_df.tail())
print("\n")

[*] Shape of dataset: (98274, 15)
         @timestamp           src_ip src_port         dst_ip dst_port  \
52984  1.666690e+09    192.168.34.11    53114  192.168.33.12        0   
15135  1.666692e+09    100.64.19.103    44618   100.64.19.22     8443   
25708  1.666692e+09    100.64.19.103    40186   100.64.19.22     6002   
89548  1.666692e+09    100.64.19.103    45640   100.64.19.22     5221   
3196   1.666542e+09  192.168.106.130     5353    224.0.0.251     5353   

                                                Protocol http_request_method  \
52984                                               HTTP                   0   
15135                                                TCP                   0   
25708                                                TCP                   0   
89548                                                TCP                   0   
3196   2022-10-24 00:18:10 ALLOW UDP 192.168.106.130 ...                 UDP   

      http_request_referrer http_response_code

In [8]:
full_df["src_port"]=  full_df["src_port"].replace(regex=",", value= "")
full_df["src_port"]=  full_df["src_port"].replace(regex=",", value= "")
full_df["dst_port"]=  full_df["dst_port"].replace(regex=",", value= "")
full_df["dst_port"]=  full_df["dst_port"].replace(regex=" ", value= "")
full_df["src_port"] = full_df["src_port"].replace(regex=" ", value="")
full_df["src_port"] = full_df["src_port"].replace(regex="dns", value="53")
full_df["dst_port"] = full_df["dst_port"].replace(regex="dns", value="53")
full_df["dst_ip"] = full_df["dst_ip"].replace(regex="\S*:+\S+", value="0")

In [10]:
print(f"[*] Shape of dataset: {full_df.shape}")
full_df.head(5)

[*] Shape of dataset: (98274, 13)


,@timestamp,src_ip,src_port,dst_ip,dst_port,Protocol,http_request_method,http_request_referrer,http_response_code,url_path,user_agent_original,sql_method,sql_query
88161,1.666592e+09,0,0,10.0.0.20,0,0,0,0,0,0,0,0,0
33113,1.666713e+09,192.168.10.2,9200,48382,192.168.10.3,TCP/UDP,0,0,0,0,0,0,0
51743,1.666704e+09,192.168.10.2,9200,47642,192.168.10.3,TCP/UDP,0,0,0,0,0,0,0
74960,1.666713e+09,192.168.10.2,9200,48382,192.168.10.3,TCP/UDP,0,0,0,0,0,0,0
58487,1.666422e+09,192.168.0.3,54468,192.168.0.2,9200,tcp,0,0,0,0,0,0,0


In [14]:
full_df.replace(to_replace=["None"], value=np.nan, inplace=True)
clean_df = full_df.fillna(str(0)) #uhm i dont knowhow to explain this, but please lmk another alternative to fix this cause :") try to run it without str and you will know what i mean"
clean_x = clean_df.iloc[:, :13].values
#clean_y = clean_df["category"].values
features = full_df.columns.values[:-1]

for label in clean_df.columns:
    for index, rows in clean_df.iterrows():
        new_ip = ""
        ip = str(rows[label])
        if re.search("\d+\.\d+\.\d+\.\d+", ip):
            octets = ip.split(".")
            for octet in octets:
                octet = octet.rjust(3,"0")
                new_ip += octet
            clean_df[label][index] = new_ip

clean_df["http_response_code"] = clean_df["http_response_code"].replace('HTTP/1.1"', value="0")
clean_df["src_ip"] = clean_df["src_ip"].replace('::1', value="0")
clean_df["dst_ip"] = clean_df["dst_ip"].replace('::1', value="0")
# clean_df["http_response_code"] = clean_df["http_response_code"].replace('HTTP/1.1"', value="0")
print("Done")

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_8768/1729664836.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[label][index] = new_ip


Done


In [ ]:
from sklearn.compose import make_column_transformer
clean_x = clean_df.iloc[:, :13] #honestly don't know if this is correct lmaoooo
#this is to iterate through the columns and convert the strings to float (passthrough means for those columns we didnt specify, leave it as it is)
column_trans = make_column_transformer((OneHotEncoder(sparse=False), ['Protocol', 'http_request_method', 'http_request_referrer', 'url_path', 'user_agent_original', 'sql_method', 'sql_query']),remainder='passthrough')
test = column_trans.fit_transform(clean_x) #this is technically our cl